In [8]:
import sys
import os
import json
sys.path.append('/home/jovyan/intake-aodn')

In [9]:
import fsspec
from fsspec.core import url_to_fs
fs = fsspec.filesystem('s3',use_listings_cache=False,anon=True,)

urls = fs.glob('s3::imos-data/IMOS/ANMN/QLD/**/*.nc')


In [10]:
len(urls)

7613

In [11]:
timeseries = list(filter(lambda x: 'aggregated_timeseries' in x, urls))

In [12]:
len(timeseries)

155

In [13]:
def process_single(url):
    import fsspec
    from kerchunk.hdf import SingleHdf5ToZarr
    
    s3_fn = 's3://' + url
    with fsspec.open(s3_fn, 
                     anon=True, 
                     mode='rb', 
                     default_fill_cache=False, 
                     default_cache_type='none') as f:
        zarr_dict = SingleHdf5ToZarr(f, s3_fn, spec=1, inline_threshold=100).translate()
        
    return zarr_dict 

In [14]:
from intake_aodn.utils import get_distributed_cluster
client = get_distributed_cluster()
client

ModuleNotFoundError: No module named 'intake_aodn'

In [15]:
from dask.distributed import PipInstall
plugin = PipInstall(packages=["kerchunk"], pip_options=["--upgrade"])
client.register_worker_plugin(plugin)

# If using a distributed cluster on EASI build eggs using "python setup.py bdist_egg" and upload to workers
# otherwise dask workers wont have code for imports
client.upload_file('/home/jovyan/intake-aodn/dist/intake_aodn-0+untagged.76.g4219cb6.dirty-py3.8.egg')

NameError: name 'client' is not defined

In [16]:
print('... using dask ...')
from dask import delayed, compute
d_process_single = delayed(process_single)
futures = [d_process_single(u) for u in timeseries]
ref_dicts = compute(futures)[0]

... using dask ...


In [18]:
for u,ref in zip(urls,ref_dicts):
    output = json.dumps(ref).encode()
    out_file = "./reffs/"+os.path.basename(u).replace('.nc','.json')
    print(out_file)
    with open(out_file,"wb") as outf:
        outf.write(output)

./reffs/IMOS_ANMN-QLD_CSTZ_20140828T090001Z_CAM050_FV01_CAM050-1408-SBE37SM-RS232-21.6_END-20150221T034501Z_C-20170214T052032Z.json
./reffs/IMOS_ANMN-QLD_CSTZ_20140828T090001Z_CAM050_FV01_CAM050-1408-SBE37SM-RS232-49.4_END-20150221T025501Z_C-20170214T052020Z.json
./reffs/IMOS_ANMN-QLD_CSTZ_20150217T013001Z_CAM050_FV01_CAM050-1502-SBE37SM-RS232-25.6_END-20150727T034501Z_C-20170217T004104Z.json
./reffs/IMOS_ANMN-QLD_CSTZ_20150219T120001Z_CAM050_FV01_CAM050-1502-SBE37SM-RS232-53.4_END-20150729T044001Z_C-20170217T004059Z.json
./reffs/IMOS_ANMN-QLD_CTZ_20140828T090001Z_CAM050_FV00_CAM050-1408-SBE37SM-RS232-21.6_END-20150221T034501Z_C-20170214T052032Z.json
./reffs/IMOS_ANMN-QLD_CTZ_20140828T090001Z_CAM050_FV00_CAM050-1408-SBE37SM-RS232-49.4_END-20150221T025501Z_C-20170214T052020Z.json
./reffs/IMOS_ANMN-QLD_CTZ_20150217T013001Z_CAM050_FV00_CAM050-1502-SBE37SM-RS232-25.6_END-20150727T034501Z_C-20170217T004104Z.json
./reffs/IMOS_ANMN-QLD_CTZ_20150219T120001Z_CAM050_FV00_CAM050-1502-SBE37SM-RS23

In [18]:
!zip -r -q nsw_refs.zip *.json

In [1]:
def open_single(fn,preprocess=None,storage_options=dict(anon=True)):
    import fsspec
    import xarray as xr
    
    mapper=fsspec.get_mapper('reference://',
                             fo=fn,
                             remote_protocol='s3',
                             remote_options=storage_options,
                            )
    ds = xr.open_zarr(mapper,chunks={}, consolidated=False, decode_times=False)   
    
    if preprocess is not None:
        ds = preprocess(ds)
    
    return ds

In [1]:
import json
with open('./reffs/IMOS_ANMN-NSW_TZ_20100504T233000Z_CH100_FV01_CH100-1005-Aqualogger-520T-77_END-20100706T013500Z_C-20160824T075053Z.json', "r") as read_file:
    ref = json.load(read_file)


In [2]:
import xarray as xr
ds = xr.open_dataset(
    "reference://", engine="zarr",
    backend_kwargs={
        "storage_options": {
            "fo": ref,
            "remote_protocol": "s3",
            "remote_options": {"anon": True}
        },
        "consolidated": False
    }
)

In [3]:
ds

<xarray.Dataset>
Dimensions:                (TIME: 17882)
Coordinates:
    LATITUDE               float64 ...
    LONGITUDE              float64 ...
    NOMINAL_DEPTH          float32 ...
  * TIME                   (TIME) datetime64[ns] 2010-05-04T23:29:59.99999667...
Data variables:
    DEPTH                  (TIME) float32 ...
    DEPTH_quality_control  (TIME) float32 ...
    TEMP                   (TIME) float32 ...
    TEMP_quality_control   (TIME) float32 ...
    TIMESERIES             float64 ...
Attributes: (12/57)
    Conventions:                      CF-1.6,IMOS-1.3
    abstract:                         NSW-IMOS Coffs Harbour 100m Mooring
    acknowledgement:                  Any users of IMOS data are required to ...
    author:                           Austin, Timothy
    author_email:                     t.austin@unsw.edu.au
    cdm_data_type:                    Station
    ...                               ...
    time_deployment_end_origin:       TimeLastInPos
    time_deployment_start:            2010-05-20T02:50:00Z
    time_deployment_start_origin:     TimeFirstInPos
    title:                            ANMN NSW-IMOS Mooring Data
    toolbox_input_file:               C:\NSW-IMOS\Processing\DATA\CH100\006_C...
    toolbox_version:                  2.5.14 - PCWIN64

In [19]:
!zip -r -q aodn_refs.zip reffs

2022-08-17 05:44:25,242 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
2022-08-17 05:44:25,244 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
